In [8]:
import sqlite3 as sl
import pandas as pd
import numpy as np

import scipy.stats as scs
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px 
import plotly.figure_factory as ff

In [9]:
def get_db_connn(path_to_db):
    conn = sl.connect(path_to_db)
    return conn


def get_table_names(conn):
    query = """
            select name from sqlite_master where type='table';
            """
    res = conn.execute(query).fetchall()
    table_names = [r[0] for r in res]
    return table_names


def get_table_column_names(conn, table_name):
    query = f"""
            pragma table_info({table_name})
            """
    res = conn.execute(query).fetchall()
    col_names = [r[1] for r in res]
    return col_names


def load_table_as_df(conn, table_name, to_display=True):
    query = f"""
            select * from {table_name}
            """
    df = pd.read_sql(query, conn)
    if to_display:
        display(df.head())
    return df

In [10]:
db_path = '../data/northWind.sqlite'
conn = get_db_connn(db_path)

In [13]:
#get_table_column_names(conn, 'Order')
query = """ pragma table_info('Order'); """
res = conn.execute(query).fetchall()
col_names = [r[1] for r in res]
print(col_names)

['Id', 'CustomerId', 'EmployeeId', 'OrderDate', 'RequiredDate', 'ShippedDate', 'ShipVia', 'Freight', 'ShipName', 'ShipAddress', 'ShipCity', 'ShipRegion', 'ShipPostalCode', 'ShipCountry']


In [14]:
orders_all_q = """SELECT * FROM 'Order'; """
orders_all_df = pd.read_sql(orders_all_q, conn)
orders_all_df.head()

,Id,CustomerId,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France
1,10249,TOMSP,6,2012-07-05,2012-08-16,2012-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany
2,10250,HANAR,4,2012-07-08,2012-08-05,2012-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,South America,05454-876,Brazil
3,10251,VICTE,3,2012-07-08,2012-08-05,2012-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,Western Europe,69004,France
4,10252,SUPRD,4,2012-07-09,2012-08-06,2012-07-11,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,Western Europe,B-6000,Belgium


In [5]:
orderdetailDF=load_table_as_df(conn, 'OrderDetail')
#get data that's needed

,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,10248/11,10248,11,14.0,12,0.0
1,10248/42,10248,42,9.8,10,0.0
2,10248/72,10248,72,34.8,5,0.0
3,10249/14,10249,14,18.6,9,0.0
4,10249/51,10249,51,42.4,40,0.0


In [22]:
orderDetail_q = """SELECT * 
                FROM OrderDetail
                JOIN 'Order'
                ON OrderDetail.OrderID = 'Order'.ID"""

In [31]:
orderDetail_df = pd.read_sql(orderDetail_q, conn)
orderDetail_df

,Id,OrderId,ProductId,UnitPrice,Quantity,Discount,Id,CustomerId,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248/11,10248,11,14.00,12,0.00,10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France
1,10248/42,10248,42,9.80,10,0.00,10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France
2,10248/72,10248,72,34.80,5,0.00,10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France
3,10249/14,10249,14,18.60,9,0.00,10249,TOMSP,6,2012-07-05,2012-08-16,2012-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany
4,10249/51,10249,51,42.40,40,0.00,10249,TOMSP,6,2012-07-05,2012-08-16,2012-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150,11077/64,11077,64,33.25,2,0.03,11077,RATTC,1,2014-05-06,2014-06-03,None,2,8.53,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,North America,87110,USA
2151,11077/66,11077,66,17.00,1,0.00,11077,RATTC,1,2014-05-06,2014-06-03,None,2,8.53,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,North America,87110,USA
2152,11077/73,11077,73,15.00,2,0.01,11077,RATTC,1,2014-05-06,2014-06-03,None,2,8.53,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,North America,87110,USA
2153,11077/75,11077,75,7.75,4,0.00,11077,RATTC,1,2014-05-06,2014-06-03,None,2,8.53,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,North America,87110,USA


In [32]:
orderDetail_df = orderDetail_df.drop(columns=['ProductId', 'UnitPrice', 'Discount', 'CustomerId', 'EmployeeId', 'OrderDate', 'RequiredDate', 'ShippedDate', 'ShipVia', 'Freight', 'ShipName', 'ShipAddress', 'ShipCity'])

In [34]:
orderDetail_df = orderDetail_df.drop(columns=['ShipRegion', 'ShipPostalCode'])

In [36]:
orderDetail_df.groupby('ShipCountry')

In [5]:
print(get_table_names(conn))

['Employee', 'Category', 'Customer', 'Shipper', 'Supplier', 'Order', 'Product', 'OrderDetail', 'CustomerCustomerDemo', 'CustomerDemographic', 'Region', 'Territory', 'EmployeeTerritory']


NameError: name 'conn' is not defined

## {Put test name here}
H0: 

HA:

In [4]:
# perform test

## Conclusion

# Question 3: {You decide}

In [2]:
# obtain necessary data

## {Put test name here}
H0: 

HA:

In [4]:
# perform test

## Conclusion

## EffectSize/PowerAnalysis/Tukey Testing (if needed)